In [3]:
import os
import sys
import json
import random
import pandas as pd
import numpy as np
from collections import deque
from src.model import BiLSTM_CRF, MakeEmbed, textCNN, EpochLogger, save
from src.dataset import Preprocessing, MakeDataset
from src.NLU import NaturalLanguageUnderstanding
from src.DM import DialogManager, E2E_dialog

class DialogSystem:
    def __init__(self):
        intent_pretrain_path = "./data/pretraining/save/1_intent_clsf_model/intent_clsf_97.217_steps_33.pt"
        entity_pretrain_path = "./data/pretraining/save/1_entity_recog_model/entity_recog_97.192_steps_7.pt"
        ood_pretrain_path = "./data/pretraining/save/1_ood_clsf_model/ood_clsf_99.724_steps_5.pt"
        chitchat_pretrain_path = "./data/pretraining/save/4_chitchat_transformer_model/chitchat_transformer_1.215381_steps_81.pt"

        self.NLU = NaturalLanguageUnderstanding()

        self.NLU.model_load(intent_pretrain_path, entity_pretrain_path, ood_pretrain_path)
        self.dm = DialogManager()

        self.e2e = E2E_dialog(self.NLU.dataset,chitchat_pretrain_path)
        self.result = {
            "DM_result": {},
            "E2E_result" : [],
            "NLG"  : "",
        }
    def task_oriented_dialog_system(self, q):
        NLU_result = self.NLU.run(q)

        system_response = self.dm.run(NLU_result)
        return system_response
    
    def chit_chat_dialog_system(self, q):
        system_response = self.e2e.predict(q)
        return system_response
    
    def ranking(self, dm_result, chitchat_result):
        self.result["DM_result"] = dm_result
        self.result["E2E_result"] = list(chitchat_result)
        dm_nlg = dm_result['NLG']
        
        if(not dm_nlg == []):
            self.result["NLG"] = dm_nlg
            return self.result
        else:
            score = chitchat_result[1]
            print(score)
            if(score>0.05):
                self.result["NLG"] = chitchat_result[0]
                return self.result
            else:
                self.result["NLG"] = "죄송합니다. 다시 말해주세요."
                return self.result
            
    def run(self, q):
        dm_result = self.task_oriented_dialog_system(q)
        chitchat_result = self.chit_chat_dialog_system(q)
        result = self.ranking(dm_result, chitchat_result)
        return result
    

In [62]:
dialogsystem = DialogSystem()

In [63]:
result=dialogsystem.run("안녕")

In [64]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': ['LOCATION^안녕'],
  'NLU': {'INTENT': 'ood', 'SLOT': ['LOCATION^안녕']},
  'NLG': ['안녕하세요! 날씨 미세먼지 여행 식당에 대해 물어봐주세요!']},
 'E2E_result': ['알면서도 데이는 인정하고 호의를 보답하세요야할개월이 망설여지야할개월이 망설여지야할개월이 망설여지야할개월이 망설여지야할 아이돌이 봅니다 인정하고 호의를 보답하세요 실망하지 그리워서 친해 수도 있겠네요 준비하면서 들어주세요 준비하면서 들어주세요 준비하면서 들어주세요 준비하면서 들어주세요겠지만 즐겁게 소액결제를 해가 깍아 설명해보세요야할 부분이니야할 부분이니야할',
  0.026331137865781784],
 'NLG': ['안녕하세요! 날씨 미세먼지 여행 식당에 대해 물어봐주세요!']}

In [65]:
result=dialogsystem.run("인천 맛집")

In [66]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': ['LOCATION^인천'],
  'NLU': {'INTENT': 'restaurant', 'SLOT': ['LOCATION^인천']},
  'NLG': ['인천 주변의 추천 식당은 OOOO입니다.']},
 'E2E_result': ['관계의 개선을 필요하겠네요하느냐에.', 0.07287415117025375],
 'NLG': ['인천 주변의 추천 식당은 OOOO입니다.']}

In [67]:
result=dialogsystem.run("미세먼지 알려줘")

In [68]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': [],
  'NLU': {'INTENT': 'dust', 'SLOT': []},
  'NLG': ['오늘 서울의 미세먼지는 좋음입니다.']},
 'E2E_result': ['마스크 쓰고 나가세요. 그렇지만 조금 힘들어져요 되어달라고 인터넷 검색가 꿰메보세요 지켜보세요 봅니다 인정하고 호의를 보답하세요 집착하는 포인트가 따라 다르겠죠 따라 다르겠죠 따라 다르겠죠 않았나봐요 조금이라도야할지 조금이라도야할지 조금이라도야할지 조금이라도야할지 조금이라도야할지 인터넷 검색가 지켜보세요 두셨네요 필요했을텐데 따라 다르겠죠 따라 다르겠죠 않았나봐요 조금이라도',
  0.07088179886341095],
 'NLG': ['오늘 서울의 미세먼지는 좋음입니다.']}

In [69]:
result=dialogsystem.run("대전")

In [70]:
result

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': ['LOCATION^대전'],
  'NLU': {'INTENT': 'ood', 'SLOT': ['LOCATION^대전']},
  'NLG': ['오늘 대전의 미세먼지는 좋음입니다.']},
 'E2E_result': ['소개팅 시켜달라고 올 기대돼 넓으신가봐요 거예요.', 0.15332584083080292],
 'NLG': ['오늘 대전의 미세먼지는 좋음입니다.']}

In [76]:
dialogsystem.run("날씨 알려줘")

{'DM_result': {'STATE': 'DS_REQ_USER_INPUT',
  'SLOT': [],
  'NLU': {'INTENT': 'weather', 'SLOT': []},
  'NLG': ['오늘 서울의 날씨는 흐림입니다.']},
 'E2E_result': ['나들이 가보세요.', 0.17587050795555115],
 'NLG': ['오늘 서울의 날씨는 흐림입니다.']}

In [72]:
result=dialogsystem.run("난 혼자인게 좋아")

0.25442710518836975


In [73]:
result

{'DM_result': {'STATE': '', 'SLOT': '', 'NLU': {}, 'NLG': []},
 'E2E_result': ['이제 일어날 시간이에요 놓아주세요.', 0.25442710518836975],
 'NLG': '이제 일어날 시간이에요 놓아주세요.'}

In [74]:
result=dialogsystem.run("꽃을 좋아해")

0.2911197543144226


In [75]:
result

{'DM_result': {'STATE': '', 'SLOT': '', 'NLU': {}, 'NLG': []},
 'E2E_result': ['좋은 회사 근처로 이사를 가보세요.', 0.2911197543144226],
 'NLG': '좋은 회사 근처로 이사를 가보세요.'}

In [77]:
dialogsystem.dm.clear()

q = ""
q = input()
while(q!="end"):
    
    result=dialogsystem.run(q)
    print(result['NLG'])
    q = input()
    
dialogsystem.dm.clear()

 안녕


['안녕하세요! 날씨 미세먼지 여행 식당에 대해 물어봐주세요!']


 미세먼지 알려줘


['오늘 서울의 미세먼지는 좋음입니다.']


 대전은


...

 대전


['오늘 대전의 미세먼지는 좋음입니다.']


 사랑해


0.08125023543834686
마음의 준비를 해봐요 사람을 변덕쟁이 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요 물어보세요 좋을 하는 수도 있겠네요


 end
